## In this Notebook we do Batch Scoring

#### You can either read the data to score from DB2Wh if it is already inserted there. Or you can read it from the local file. Please follow the instruction in the lab to determine which way you need to read this data. Here we have examples of both approaches.

#### The cell below shows how to read the data to score from Db2Wh

In [96]:


# import dsx_core_utils, requests, os, io
# from pyspark.sql import SparkSession
# # Add asset from remote connection
# df2 = None
# dataSet = dsx_core_utils.get_remote_data_set_info('CUST_DAILY_TEST00')
# dataSource = dsx_core_utils.get_data_source_info(dataSet['datasource'])
# sparkSession = SparkSession(sc).builder.getOrCreate()
# # Load JDBC data to Spark dataframe
# dbTableOrQuery = '"' + (dataSet['schema'] + '"."' if(len(dataSet['schema'].strip()) != 0) else '') + dataSet['table'] + '"'
# if (dataSet['query']):
#     dbTableOrQuery = "(" + dataSet['query'] + ") TBL"
# df2 = sparkSession.read.format("jdbc").option("url", dataSource['URL']).option("dbtable", dbTableOrQuery).option("user",dataSource['user']).option("password",dataSource['password']).load()
# df2.show(5)




+-----+---+------+------+--------+---------+--------+---------+------------+-------------+-----+-------+---------+-------------+--------------------+-----+--------+
|PHASE| ID|GENDER|STATUS|CHILDREN|ESTINCOME|CAROWNER|      AGE|LONGDISTANCE|INTERNATIONAL|LOCAL|DROPPED|PAYMETHOD|LOCALBILLTYPE|LONGDISTANCEBILLTYPE|USAGE|RATEPLAN|
+-----+---+------+------+--------+---------+--------+---------+------------+-------------+-----+-------+---------+-------------+--------------------+-----+--------+
|Adult| 14|     F|     M|       2|  52004.8|       N|    25.14|           5|            0|   23|      0|       CH|       Budget|      Intnl_discount|   28|       1|
|Adult| 18|     M|     M|       1|  75004.5|       N|     64.8|          26|            0|   32|      0|       CC|       Budget|      Intnl_discount|   58|       1|
|Adult| 21|     M|     M|       0|  19749.3|       N|60.366667|          20|            0|   13|      0|       CC|       Budget|            Standard|   34|       3|
|Adult| 22

#### The cell below shows the approach to read the scoring data from a local csv file

In [2]:


import os
from pyspark.sql import SQLContext
# Add asset from file system
df_data_1 = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/CustomerChurnScoringData.csv', header='true', inferSchema = 'true')
df_data_1.show(5)

df2 = df_data_1





+-----+---+------+------+--------+---------+--------+---------+------------+-------------+-----+-------+---------+-------------+--------------------+-----+--------+
|PHASE| ID|GENDER|STATUS|CHILDREN|ESTINCOME|CAROWNER|      AGE|LONGDISTANCE|INTERNATIONAL|LOCAL|DROPPED|PAYMETHOD|LOCALBILLTYPE|LONGDISTANCEBILLTYPE|USAGE|RATEPLAN|
+-----+---+------+------+--------+---------+--------+---------+------------+-------------+-----+-------+---------+-------------+--------------------+-----+--------+
|Adult| 14|     F|     M|       2|  52004.8|       N|    25.14|           5|            0|   23|      0|       CH|       Budget|      Intnl_discount|   28|       1|
|Adult| 18|     M|     M|       1|  75004.5|       N|     64.8|          26|            0|   32|      0|       CC|       Budget|      Intnl_discount|   58|       1|
|Adult| 21|     M|     M|       0|  19749.3|       N|60.366667|          20|            0|   13|      0|       CC|       Budget|            Standard|   34|       3|
|Adult| 22

#### In the next cell change the data frame name in the right side to same as the name you got previously either reading the scoring data from DB2Wh or from local file

In [3]:
churn_data_daily = df2

In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [10]:
wml_credentials = {
    "url": "<the url of cp4d without the port>"",
    "username": "<your user id with administrator permission>",
    "password": ">your password",
    "instance_id": "icp"               
}

wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [117]:
wml_client.deployments.list()

------------------------------------  -------------------------------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                                         TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
468a584c-bafe-4101-8caf-e391ba8588a4  Customer Churn ICP4D function deployment v6  online  DEPLOY_SUCCESS  2019-09-17T21:41:29.363Z  n/a                function
36ea79b2-4e4e-4a72-937c-703cd0535084  churn-model-in-cp4d-wml-dep1                 online  DEPLOY_SUCCESS  2019-09-17T19:01:59.099Z  mllib-2.3          model
6e3470a3-4a95-4a42-a050-121a4a51ac13  GermanCreditRiskModelICP                     online  DEPLOY_SUCCESS  2019-09-17T16:32:06.303Z  mllib-2.3          model
fc8d97e7-584b-498b-8f63-d6409ec08b78  HouseCreditRiskModelICP                      online  DEPLOY_SUCCESS  2019-09-17T15:58:22.893Z  scikit-learn-0.19  model
442be063-8d3b-4a03-8e65-47fa8bc8e

#### Identify the right deployment id from the above list corresponding to the deployment to which you want to send the scoring request. 
#### Use the same deployment id to get deployment details in the next cell

In [118]:
deployment_details = wml_client.deployments.get_details('468a584c-bafe-4101-8caf-e391ba8588a4')

In [119]:
icp4d_churn_scoring_url = deployment_details['entity']['scoring_url']
print(icp4d_churn_scoring_url)

https://52.116.34.23:31843/v3/scoring/online/468a584c-bafe-4101-8caf-e391ba8588a4


#### Now create a Lambda function so that the scoring URL can be invoked for multiple records in parallel

In [120]:
def wml_score(score_url, score_vals, score_id):
       from watson_machine_learning_client import WatsonMachineLearningAPIClient
       wml_credentials_wrks={
              "url": "https://52.116.34.23",
              "username": "test1",
              "password": "ibm123",
              "instance_id": "icp"    
       }
       wmlclient0 = WatsonMachineLearningAPIClient(wml_credentials_wrks)
       fields = ["GENDER", "STATUS", "CAROWNER", "PAYMETHOD", "LOCALBILLTYPE", \
          "LONGDISTANCEBILLTYPE", "PHASE", "CHILDREN", "ESTINCOME", "AGE", "LONGDISTANCE", "INTERNATIONAL", "LOCAL",\
          "DROPPED","USAGE","RATEPLAN"]
       vals = score_vals
       payload_scoring0 = {"fields": fields,"values": [vals]}
       scoring_response0 = wmlclient0.deployments.score(score_url, payload_scoring0, score_id)
       return scoring_response0

In [121]:
churn_data_daily.createOrReplaceTempView("scoringT")

In [122]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()

In [123]:
import random
indx = 'rfc' + str(random.randint(10000,20000))

#### Get a Random set of Scoring Data each time you run. You can change the split ration to increase or decrease the number of scoring records

In [124]:
scoreinp = spark.sql("select GENDER, STATUS, CAROWNER, PAYMETHOD, LOCALBILLTYPE, LONGDISTANCEBILLTYPE, PHASE, ID, CHILDREN,"\
                     "ESTINCOME, AGE, DROPPED, RATEPLAN, cast(ID as String) as str_id, cast(longdistance as double) as LONGDISTANCE, "\
                     "cast(international as double) as INTERNATIONAL, cast(local as double) as LOCAL, cast(usage as double) as USAGE "\
                     "from scoringT ").repartition(5)

In [125]:
scoreinp.printSchema()

root
 |-- GENDER: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- CAROWNER: string (nullable = true)
 |-- PAYMETHOD: string (nullable = true)
 |-- LOCALBILLTYPE: string (nullable = true)
 |-- LONGDISTANCEBILLTYPE: string (nullable = true)
 |-- PHASE: string (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CHILDREN: integer (nullable = true)
 |-- ESTINCOME: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DROPPED: integer (nullable = true)
 |-- RATEPLAN: integer (nullable = true)
 |-- str_id: string (nullable = true)
 |-- LONGDISTANCE: double (nullable = true)
 |-- INTERNATIONAL: double (nullable = true)
 |-- LOCAL: double (nullable = true)
 |-- USAGE: double (nullable = true)



In [126]:
scoreRes = scoreinp.rdd.map(lambda line : tuple([line['ID'], \
                wml_score(icp4d_churn_scoring_url, \
                [line['GENDER'], line['STATUS'],line['CAROWNER'], line['PAYMETHOD'], line['LOCALBILLTYPE'], line['LONGDISTANCEBILLTYPE'], \
                line['PHASE'], line['CHILDREN'],line['ESTINCOME'], line['AGE'], line['LONGDISTANCE'], line['INTERNATIONAL'], \
                line['LOCAL'], line['DROPPED'],line['USAGE'], line['RATEPLAN']], \
                indx+line['str_id'])])).toDF(['ID', 'pred']).cache()

In [127]:
scoreRes.count()

533

In [128]:
scoreRes.printSchema()

root
 |-- ID: long (nullable = true)
 |-- pred: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: string (containsNull = true)



In [115]:
scoreRes.createOrReplaceTempView("scoresT")

In [116]:
spark.sql("select ID, explode(pred.values) as scordarr from scoresT").show(10, False)

+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ID  |scordarr                                                                                                                                                                                                                                                                                                                            |
+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|121

In [79]:
scorefdf2 = spark.sql("select ID, explode(pred.values) as scordarr from scoresT").cache()

In [80]:
scorefdf2.createOrReplaceTempView("scores3T")

In [81]:
spark.sql("select ID, substr(split(scordarr, ',')[size(split(scordarr, ','))-1],1,2) as ChurnFlg from scores3T").show(20, False)

+----+--------+
|ID  |ChurnFlg|
+----+--------+
|1215| T      |
|530 | T      |
|2984| F      |
|419 | T      |
|1135| T      |
|317 | T      |
|1408| T      |
|3681| T      |
|2718| F      |
|1851| T      |
|1312| T      |
|2279| F      |
|1799| F      |
|2932| F      |
|3657| T      |
|2644| F      |
|1184| F      |
|518 | F      |
|3009| T      |
|1999| T      |
+----+--------+
only showing top 20 rows



In [82]:
churnDf = spark.sql("select ID, substr(split(scordarr, ',')[size(split(scordarr, ','))-1],1,2) as ChurnFlg from scores3T")

In [83]:
churnDf.printSchema()

root
 |-- ID: long (nullable = true)
 |-- ChurnFlg: string (nullable = true)



In [84]:
churnDf.count()

533

In [32]:
jdbcuri = "jdbc:db2://10.187.215.33:31959/BLUDB"

properties = {
    "user": "user999",
    "password": "bGSgd%77k7VZ1**@",
    "driver": "com.ibm.db2.jcc.DB2Driver",
    "sslConnection":"false"
}

TABLE_NAME = "churnscores"

In [33]:
churnDf.write.jdbc(url=jdbcuri, table=TABLE_NAME, mode="overwrite", properties=properties)

In [34]:
rdDb2df = spark.read.jdbc(url=jdbcuri, table=TABLE_NAME, properties=properties)

In [35]:
rdDb2df.count()

2

In [36]:
rdDb2df.show()

+----+--------+
|  ID|ChurnFlg|
+----+--------+
|1215|       T|
| 530|       T|
+----+--------+

